<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import urllib.request
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
import time
import sys
import os
import pandas as pd
from functools import reduce
from operator import itemgetter
import itertools
import lxml.html
from lxml import etree
import re
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [3]:
# A Team class which tracks team stats and possessions throughout games
class Team:
    def __init__(self, name, home_court, p_active, p_start):
        self.name = name
        self.home_court = home_court
        self.p_active = p_active
        self.p_start = p_start
        self.lineup = p_start
        self.score = 0
        self.margin = 0
        self.team_fouls = 0
        self.penalty = False
        self.technicals = 0
        self.flagrants = 0
        self.to_forced = 0
        self.to_unforced = 0
        self.time_off = 0
        self.time_def = 0
        self.two_pt_attempts = 0
        self.three_pt_attempts = 0
        self.two_pt_made = 0
        self.three_pt_made = 0
        if home_court:
            self.poss = 'OFF'
        else:
            self.poss = 'DEF'
            
    def StartQuarterLineup(self, quarter):
        if quarter <= 0 or quarter > 4:
            print('*** Use quarter index 1 - 4 to choose starting lineup! ***')
            sys.exit(1)
        self.lineup = self.p_start[quarter-1]
            

In [2]:
# Divide files into manageable chunks
def DivideFileChunks(l, n):
    for i in range(0, len(l), n):  
        yield l[i:i + n]


In [4]:
# Function for finding which team a player belongs to
def WhichTeam(player, home_lineup, vis_lineup):
    # Return 0 for home team
    for p in home_lineup:
        if player in p:
            return 0
    # Return 1 for visiting team
    for p in vis_lineup:
        if player in p:
            return 1

In [5]:
# Parse the text data of plays and extract the 
# player substitution data. Use this data (along with
# fuzzy text matching) to swap players in the team lineup
def PlayerSubstitution(play, team_lineup, current_lineup):
    result = re.search('SUB: (.*) FOR', play)
    if result.group(1).split()[-1].lower() == 'jr.':
        sub = ' '.join(result.group(1).split()[-2:])
    else:
        sub = result.group(1).split()[-1]
        
    result = re.search('FOR (.*)', play)
    if result.group(1).split()[-1].lower() == 'jr.':
        op = ' '.join(result.group(1).split()[-2:])
    else:
        op = result.group(1).split()[-1]
    
    #print(team_lineup)
    sub_name = ''
    for player in team_lineup:
        if sub in player:
            sub_name = player
            break
    if not sub_name:
        fuzzy_list = []
        for x in team_lineup: 
            fuzzy_list.append(fuzz.token_sort_ratio(sub, x))
        matches = [x for _,x in sorted(zip(fuzzy_list, team_lineup), reverse=True)]
        sub_name = matches[0]
    
    substituted = False
    for i,player in enumerate(current_lineup):
        if op in player:
            current_lineup[i] = sub_name
            substituted = True
            #print('Replaced', player, 'with', sub_name)
    if not substituted:
        fuzzy_list = []
        for x in current_lineup: 
            fuzzy_list.append(fuzz.token_sort_ratio(op, x))
        index, match = max(enumerate(fuzzy_list), key=itemgetter(1))
        current_lineup[index] = sub_name
    
    return np.array(current_lineup)


In [6]:
# For each NBA game in the set, grab the starting lineup for each
# quarter (for both teams) and fetch the play-by-play table for each game.
# Process the play-by-play data to track cumulative team stats (throughout the game)
# and to track the substitutions/lineup changes throughout the game.
def FetchPlayByPlayTables(urls, games, years):
    # Launch a headless player instance
    opt = FirefoxOptions()
    opt.add_argument("--headless")
    driver = webdriver.Firefox(options=opt)
    
    # Times (in seconds * 10) to start off with when 
    # finding the starting lineup for each quarter
    q_starts = [0, 7210, 14410, 21610]
    q_ends = [240, 7440, 14640, 21840]
    # The amount of time to shift the slider
    # if the starting lineup contains less than 5 players 
    # at the start of the quarter
    window_expand = 20 # sec*10
    
    # Loop over game URLs and process each game
    arr = []
    for i,url in enumerate(urls):
        year = years[i]
        game = games[i]
        sys.stdout.write("\rOn Game %i of the %i season..." % (game,year))
        sys.stdout.flush()
        
        # Extract the roster of active players used in this game (for each team)
        driver.get(url)
        wait = WebDriverWait(driver, 30)
        retries = 1
        while retries <= 3:
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, "//*[@rows='linescores.htm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr/td[@class='player']")))
                break
            except TimeoutException:
                print('\nRefreshing lineup page due to timeout (retry #', retries,')...')
                driver.refresh()
                retries += 1

        root = lxml.html.fromstring(driver.page_source)
        team_txt = root.xpath("//*[@class='game-summary__info-table']/tbody/tr/td[contains(@class, 'team-name')]//text()")
        indices = np.unique(np.array(team_txt), return_index=True)[1]
        team_abr = [team_txt[index] for index in sorted(indices)]
        vteam = team_abr[0]
        hteam = team_abr[1]
        
        htm_actives = []
        vtm_actives = []
        vtm_rows = root.xpath("//*[@rows='linescores.vtm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr")
        htm_rows = root.xpath("//*[@rows='linescores.htm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr")
        for row in vtm_rows:
            td = row.xpath("./td[@class='player']//text()")
            td = [re.sub('\n +', '', x) for x in td]
            td = [x for x in td if x != '' and x != '\n']
            vtm_actives.append(td[0].lstrip().rstrip())
        for row in htm_rows:
            td = row.xpath("./td[@class='player']//text()")
            td = [re.sub('\n +', '', x) for x in td]
            td = [x for x in td if x != '' and x != '\n']
            htm_actives.append(td[0].lstrip().rstrip())
        #print(htm_actives, vtm_actives)
        
        # Adjust the time slider on the game webpage to extract
        # the starting lineup for each quarter 
        # (needed to track player substitutions)
        htm_qstart_lineups = []
        vtm_qstart_lineups = []
        #print('Grabbing starting lineup for each quarter')
        for j in np.arange(len(q_starts)):
            # Grabbing starting/active player info for home and away teams (for THIS game)
            #print('On Quarter', i+1)
            driver.get(url + '?StartRange={0}&EndRange={1}&RangeType=2'.format(str(q_starts[j]), str(q_ends[j])))
            retries = 1
            # Extends the search window at start of quarter to retries*window_expand
            while retries <= 10:
                try:
                    wait.until(EC.presence_of_element_located((By.XPATH, "//*[@rows='linescores.htm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr/td[@class='player']")))
                    break
                except TimeoutException:
                    #print('Refreshing lineup page due to timeout (retry #', retries,')...')
                    #driver.refresh()
                    print('\nRefreshing lineup page due to timeout (extending search window by', retries*window_expand/10.,'seconds)...')
                    driver.get(url + '?StartRange={0}&EndRange={1}&RangeType=2'.format(str(q_starts[j]), str(q_ends[j]+retries*window_expand)))
                    retries += 1

            root = lxml.html.fromstring(driver.page_source)
            vtm_rows = root.xpath("//*[@rows='linescores.vtm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr")
            htm_rows = root.xpath("//*[@rows='linescores.htm.datatable']//div[@class='nba-stat-table']/div[@class='nba-stat-table__overflow']/table/tbody/tr")

            vtm_arr = []
            htm_arr = []
            for row in vtm_rows:
                td = row.xpath("./td[@class='player']//text()")
                td = [re.sub('\n +', '', x) for x in td]
                td = [x for x in td if x != '' and x != '\n']
                vtm_arr.append(td[0].lstrip().rstrip())
                #if len(td) == 2:
                #    vtm_sl.append(td[0].lstrip().rstrip())
            for row in htm_rows:
                td = row.xpath("./td[@class='player']//text()")
                td = [re.sub('\n +', '', x) for x in td]
                td = [x for x in td if x != '' and x != '\n']
                htm_arr.append(td[0].lstrip().rstrip())
                #if len(td) == 2:
                #    htm_sl.append(td[0].lstrip().rstrip())
            vtm_qstart_lineups.append(vtm_arr)
            htm_qstart_lineups.append(htm_arr)
            if len(htm_arr) != 5 or len(vtm_arr) != 5:
                print('\n*** Got', len(htm_arr), 'man lineup and', len(vtm_arr), 'man lineup for Quarter', j+1, '***')
        
        #print(htm_qstart_lineups, vtm_qstart_lineups)
        
        
        # Grab play-by-play data for this game
        driver.get(url+'playbyplay/')
        retries = 1
        while retries <= 3:
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, "//*[@class='boxscore-pbp__inner']//table/tbody/tr//td[@id='qtr4']")))
                break
            except TimeoutException:
                print('\nRefreshing play-by-play page due to timeout (retry #', retries,')...')
                driver.refresh()
                retries += 1

        root = lxml.html.fromstring(driver.page_source)
        rows = root.xpath("//*[@class='boxscore-pbp__inner']//table/tbody/tr")
        
        # Instantiate Team class for home and visiting teams
        #print(list(np.unique(np.array(htm_actives))), list(np.unique(np.array(vtm_actives))))
        htm = Team(hteam, True, list(np.unique(np.array(htm_actives))), htm_qstart_lineups)
        vtm = Team(vteam, False, list(np.unique(np.array(vtm_actives))), vtm_qstart_lineups)
        
        # Each row is a play in the play-by-play table
        quarter = 0
        t = 0
        if len(rows) == 0:
            print('*** Only found', len(rows), 'rows in game table ***')
        for row in rows:
            home_play = ''
            vis_play = ''
            change_poss = False
            
            td = row.xpath(".//td[contains(@class, 'start-period')]//text()")
            if td:
                td = [re.sub('\n +', '', x) for x in td]
                parr = [x for x in td if x != '' and x != '\n']
                if len(parr) == 1:
                    if 'Start of Q' in parr[0]:
                        quarter += 1
                        htm.team_fouls = 0
                        vtm.team_fouls = 0
                        htm.penalty = False
                        vtm.penalty = False
                        htm.StartQuarterLineup(quarter)
                        vtm.StartQuarterLineup(quarter)
                        #print('Starting lineup for Quarter', quarter, ':\n', htm.lineup, vtm.lineup)
                        #print('On Quarter', quarter, '...')
                        continue
            
            td = row.xpath("./td[contains(@class, 'team') and contains(@class, 'vtm')]//text()")
            td = [re.sub('\n +', '', x) for x in td]
            larr = [x for x in td if x != '' and x != '\n']
            td = row.xpath("./td[contains(@class, 'play') and contains(@class, 'status')]//text()")
            td = [re.sub('\n +', '', x) for x in td]
            marr = [x for x in td if x != '' and x != '\n']
            td = row.xpath("./td[contains(@class, 'team') and contains(@class, 'htm')]//text()")
            td = [re.sub('\n +', '', x) for x in td]
            rarr = [x for x in td if x != '' and x != '\n']
                        
            # Grab the time (and score if appropriate) from middle column
            if len(marr) == 1:
                qclock = marr[0]
            elif len(marr) == 2:
                qclock = marr[0]
                score = marr[1]
                htm.score = int(score.split(' - ')[-1])
                vtm.score = int(score.split(' - ')[0])
            else:
                continue
            
            # Extract time of the play in seconds (since start of game)
            previous_t = t
            t = (quarter-1)*12.*60. + 12.*60.-(float(qclock.split(':')[0])*60.+float(qclock.split(':')[1]))
            
            # Get play description for visiting and home teams
            if len(larr) == 1:
                vis_play = larr[0]
            if len(rarr) == 1:
                home_play = rarr[0]
            
            # Proceed to next play if nothing happened (special case)
            if home_play == '' and vis_play == '':
                continue
            
            # Change the current lineups if there are player substitutions
            if 'sub:' in home_play.lower():
                htm.lineup = list(np.unique(PlayerSubstitution(home_play, htm.p_active, htm.lineup)))
            if 'sub:' in vis_play.lower():
                vtm.lineup = list(np.unique(PlayerSubstitution(vis_play, vtm.p_active, vtm.lineup)))
            
            # Change of possession due to jump ball scenario
            if 'jump ball' in home_play.lower() or 'jump ball' in vis_play.lower():
                if 'jump ball' in home_play.lower():
                    jump_play = home_play
                else:
                    jump_play = vis_play
                
                #Format: 'Jump Ball Nance Jr. vs. Clark: Tip to Clarkson'
                result = re.search('Tip to (.*)', jump_play)
                if result:
                    player = result.group(1).lstrip().rstrip().split()[-1]
                    if not WhichTeam(player, htm.p_active, vtm.p_active):
                        htm.poss = 'OFF'
                        vtm.poss = 'DEF'
                    else:
                        vtm.poss = 'OFF'
                        htm.poss = 'DEF'
                #continue
                #else:
                    #print("*** Couldn't get jump ball result ***")
                    #break
            
            # Use shots taken and turnovers to determine whether
            # a team is on offense or defense for this possession.
            # Automatically mark a change in possession if the shot
            # is made, the last free throw shot is made, or there is a turnover.
            score_types = ['shot', 'layup', 'dunk', 'free throw']
            unforced_to_types = ['travel', 'bad pass', 'out of bounds']
            forced_to_types = ['lost ball', 'shot clock', 'off.foul', 'charge']
            off_types = score_types + unforced_to_types + forced_to_types
            
            if any(x in home_play.lower() for x in off_types):
                htm.poss = 'OFF'
                vtm.poss = 'DEF'
                if any(x in home_play.lower() for x in score_types) \
                and 'miss' not in home_play.lower() and 'free throw' not in home_play.lower():
                    #print('Home team made a shot')
                    change_poss = True
                    if '3pt' in home_play.lower():
                        htm.three_pt_attempts += 1
                        htm.three_pt_made += 1
                    else:
                        htm.two_pt_attempts += 1
                        htm.two_pt_made += 1
                elif any(x in home_play.lower() for x in score_types) \
                and 'miss' in home_play.lower() and 'free throw' not in home_play.lower():
                    if '3pt' in home_play.lower():
                        htm.three_pt_attempts += 1
                    else:
                        htm.two_pt_attempts += 1
                elif 'free throw' in home_play.lower():
                    if 'technical' not in home_play.lower() and 'flagrant' not in home_play.lower():
                        ft = re.findall(r'[0-9]+', home_play.lower())
                        if ft[0] == ft[1] and 'miss' not in home_play.lower():
                            change_poss = True
                elif any(x in home_play.lower() for x in unforced_to_types):
                    change_poss = True
                    htm.to_unforced += 1
                    #print('Unforced turnover on play:', home_play.lower())
                elif any(x in home_play.lower() for x in forced_to_types):
                    change_poss = True
                    htm.to_forced += 1
                    #print('Forced turnover on play:', home_play.lower())                    
            elif any(x in vis_play.lower() for x in off_types):
                vtm.poss = 'OFF'
                htm.poss = 'DEF'
                if any(x in vis_play.lower() for x in score_types) \
                and 'miss' not in vis_play.lower() and 'free throw' not in vis_play.lower():
                    #print('Visiting team made a shot')
                    change_poss = True
                    if '3pt' in vis_play.lower():
                        vtm.three_pt_attempts += 1
                        vtm.three_pt_made += 1
                    else:
                        vtm.two_pt_attempts += 1
                        vtm.two_pt_made += 1
                elif any(x in vis_play.lower() for x in score_types) \
                and 'miss' in vis_play.lower() and 'free throw' not in vis_play.lower():
                    if '3pt' in vis_play.lower():
                        vtm.three_pt_attempts += 1
                    else:
                        vtm.two_pt_attempts += 1
                elif 'free throw' in vis_play.lower():
                    if 'technical' not in vis_play.lower() and 'flagrant' not in vis_play.lower():
                        ft = re.findall(r'[0-9]+', vis_play.lower())
                        if ft[0] == ft[1] and 'miss' not in vis_play.lower():
                            change_poss = True
                elif any(x in vis_play.lower() for x in unforced_to_types):
                    change_poss = True
                    vtm.to_unforced += 1
                    #print('Unforced turnover on play:', vis_play.lower())
                elif any(x in vis_play.lower() for x in forced_to_types):
                    change_poss = True
                    vtm.to_forced += 1
                    #print('Forced turnover on play:', vis_play.lower())

            # Mark change of possession due to rebound, but only
            # if the rebounding team is currently on defense
            if 'rebound' in home_play.lower():
                if htm.poss == 'DEF':
                    change_poss = True
            elif 'rebound' in vis_play.lower():
                if vtm.poss == 'DEF':
                    change_poss = True
            
            # Track different types of fouls
            foul_types = ['p.foul', 's.foul', 'l.b.foul', 'block foul']
            if any(x in home_play.lower() for x in foul_types):
                htm.team_fouls += 1
            if any(x in vis_play.lower() for x in foul_types):
                vtm.team_fouls += 1
            if 'pn)' in home_play.lower():
                htm.penalty = True
            if 'pn)' in vis_play.lower():
                vtm.penalty = True
            if 't.foul' in home_play.lower():
                htm.technicals += 1
            if 't.foul' in vis_play.lower():
                vtm.technicals += 1
            if 'flagrant' in home_play.lower():
                htm.flagrants += 1
            if 'flagrant' in vis_play.lower():
                vtm.flagrants += 1
            
            if 'Go to top' in home_play or 'Go to top' in vis_play:
                home_play = 'EOQ'
                vis_play = 'EOQ'
                
            #print('At time of', t, 'seconds, score is H:', hscore, ', V:', vscore)
            #print('Home Play:', home_play)
            #print('Visiting Play:', vis_play)
            
            # Update calculations
            htm.margin = htm.score - vtm.score
            vtm.margin = -htm.margin
            if htm.poss == 'OFF':
                htm.time_off += (t - previous_t)
                vtm.time_def = htm.time_off
            else:
                htm.time_def += (t - previous_t)
                vtm.time_off = htm.time_def
            
            game_arr = [year, game, htm.name, vtm.name, quarter, t, htm.score, vtm.score, htm.margin, vtm.margin, htm.poss, vtm.poss, htm.time_off, vtm.time_off, htm.time_def, vtm.time_def, ','.join(htm.lineup), ','.join(vtm.lineup), home_play, vis_play, htm.team_fouls, vtm.team_fouls, htm.penalty, vtm.penalty, htm.technicals, vtm.technicals, htm.flagrants, vtm.flagrants, htm.two_pt_attempts, vtm.two_pt_attempts, htm.three_pt_attempts, vtm.three_pt_attempts, htm.two_pt_made, vtm.two_pt_made, htm.three_pt_made, vtm.three_pt_made]
            #print(game_arr)
            arr.append(game_arr)
            
            # Change of possession
            if change_poss:
                if htm.poss == 'OFF':
                    htm.time_off = 0
                    vtm.time_def = 0
                    htm.poss = 'DEF'
                    vtm.poss = 'OFF'
                else:
                    htm.time_def = 0
                    vtm.time_off = 0
                    htm.poss = 'OFF'
                    vtm.poss = 'DEF'
            
            if t == 2880.:
                break

    driver.quit()
    return np.array(arr)
    

In [7]:
# Specify the starting season, starting game (for that season),
# and file chunk size to use for scraping PBP data
start_year = 19 # for season (start_year-1 -> start_year)
start_game = 1
chunk_size = 10

# Create arrays of years and year strings for URL construction
ya = [str(n).zfill(2) for n in range(start_year-1, 19)]
yb = [str(n).zfill(2) for n in range(start_year, 20)]
years = [int("20"+y) for y in yb]

# Create array of game numbers for URL construction
# and divide these games into manageable chunks
gn = [str(n).zfill(4) for n in range(start_game, 1231)]
games = [int(x) for x in gn]
file_chunks = list(DivideFileChunks(gn, chunk_size))
#print(file_chunks)

# Scrape PBP data and append dataframes to the CSV file for that season
for i,year in enumerate(years):
    print('---> Processing play-by-play for the', year-1, '-', year, 'season --->')
    filename = 'NBA_PBP_Data_'+str(year-1)+'_'+str(year)+'.csv'
    for j,chunk in enumerate(file_chunks):
        games = [int(g) for g in chunk]
        #urls = [ "https://stats.nba.com/game/0021600014/".format(ya[i], g) for g in chunk ]
        urls = [ "https://stats.nba.com/game/002{0}0{1}/".format(ya[i], g) for g in chunk ]
        #print(chunk, games)
        start = time.time()
        np_arr = FetchPlayByPlayTables(urls, games, np.repeat(np.array([year]), chunk_size))
        end = time.time()
        print('\nTook', end-start, 'seconds to process games', games[0], '-', games[-1], '.')
        df = pd.DataFrame(np_arr, columns=['year', 'game', 'home_team', 'vis_team', 'Q', 'time_sec', 'ht_score', 'vt_score', 'ht_margin', 'vt_margin', 'ht_poss', 'vt_poss', 'ht_time_off', 'vt_time_off', 'ht_time_def', 'vt_time_def', 'ht_lineup', 'vt_lineup', 'ht_play', 'vt_play', 'ht_fouls', 'vt_fouls', 'ht_penalty', 'vt_penalty', 'ht_technicals', 'vt_technicals', 'ht_flagrants', 'vt_flagrants', 'ht_2PTA', 'vt_2PTA', 'ht_3PTA', 'vt_3PTA', 'ht_2PTM', 'vt_2PTM', 'ht_3PTM', 'vt_3PTM'])
        print('Writing data to file:', filename)
        if not os.path.isfile(filename):
            df.to_csv(filename)
        else:
            df.to_csv(filename, mode='a', header=False)
        time.sleep(3)


---> Processing play-by-play for the 2018 - 2019 season --->
On Game 1 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 10 of the 2019 season...
Took 306.63106322288513 seconds to process games 1 - 10 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 12 of the 2019 season...
*** Got 6 man lineup and 5 man lineup for Quarter 2 ***
On Game 15 of the 2019 season...
*** Got 6 man lineup and 5 man lineup for Quarter 3 ***
On Game 18 of the 2019 season...
*** Got 6 man lineup and 5 man lineup for Quarter 4 ***
On Game 20 of the 2019 season...
Took 332.3637707233429 seconds to process games 11 - 20 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 23 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 28 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 30 of the 2019 season...
Took 383.236361

On Game 365 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 367 of the 2019 season...
*** Got 5 man lineup and 6 man lineup for Quarter 2 ***
On Game 370 of the 2019 season...
Took 274.9775643348694 seconds to process games 361 - 370 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 373 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 376 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 380 of the 2019 season...
Took 311.6559669971466 seconds to process games 371 - 380 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 384 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 390 of the 2019 season...
Took 289.5505337715149 seconds to process games 381 - 390 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 400 of th

On Game 709 of the 2019 season...
*** Got 5 man lineup and 6 man lineup for Quarter 4 ***
On Game 710 of the 2019 season...
Took 222.115149974823 seconds to process games 701 - 710 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 720 of the 2019 season...
Took 248.3893918991089 seconds to process games 711 - 720 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 723 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 727 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 730 of the 2019 season...
Took 288.24982595443726 seconds to process games 721 - 730 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 740 of the 2019 season...
Took 221.20634198188782 seconds to process games 731 - 740 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 744 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 s

On Game 1030 of the 2019 season...
Took 406.7304229736328 seconds to process games 1021 - 1030 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 1031 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...

Refreshing lineup page due to timeout (extending search window by 4.0 seconds)...
On Game 1034 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 1035 of the 2019 season...
Refreshing lineup page due to timeout (extending search window by 2.0 seconds)...
On Game 1040 of the 2019 season...
Took 337.8909869194031 seconds to process games 1031 - 1040 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 1046 of the 2019 season...
*** Got 5 man lineup and 6 man lineup for Quarter 2 ***
On Game 1050 of the 2019 season...
Took 246.28453016281128 seconds to process games 1041 - 1050 .
Writing data to file: NBA_PBP_Data_2018_2019.csv
On Game 1052 of the 2019 season...
*